Гид по библиотеке **natasha** написала Глуходед Анастасия (324 группа). 
Текст для примера взят из романа Джорджа Оруэлла "1984".

Источники:
https://github.com/natasha/natasha

Библиотека **natasha** решает базовые задачи НЛП (нейролингвистического программирования) естественного русского языка: токенизация, сегментация предложений на токены и предложения, встраивание слов, маркировка морфологии, лемматизация, нормализация фраз, синтаксический анализ, маркировка NER, извлечение фактов.

Давайте установим ее и посмотрим, как она работает.

In [ ]:
pip install natasha

Экспортируем библиотеку и пропишем все необходимые строки для обработки текста.

In [2]:
from natasha import (
    Segmenter, #сегментация текста на слова и предложения
    MorphVocab, #лемматизация слов
    NewsEmbedding,
    NewsMorphTagger, #морфологический разбор
    NewsSyntaxParser, #синтаксический разбор
    NewsNERTagger, #извлечение стандартных именованных объектов
    #(имен, местоположений, организаций)
    
    PER,
    NamesExtractor, #извлечение имен (фамилии, имени, отсчества)
    #Пример: «Анастасия Александровна Глуходед» - 
    #{first: Анастасия, last: Глуходед, middle: Александровна}
    DatesExtractor, #извлечение дат (в различном формате)
    #Пример: "30.09.2021" - {year=2021, month=09, day=30}
    MoneyExtractor, #извлечение денежных единиц, валюты
    #Пример: "1000 RUB" - {amount=1000,currency='RUB'}
    AddrExtractor, #извлечение адресов
    
    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
noney_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

Введем текст, с которым будем работать:

In [3]:
text = '4 апреля 1984 года. Вчера в кино. Сплошь военные фильмы. Один очень хороший где-то в Средиземном море бомбят судно с беженцами. Публику забавляют кадры где пробует уплыть громадный толстенный мужчина а его преследует вертолет. сперва мы видим как он по-дельфиньи бултыхается в воде потом видим его с вертолета через прицел потом он весь продырявлен и море вокруг него розовое и сразу тонет словно через дыры набрал воды, когда он пошел на дно зрители загоготали. Потом шлюпка полная детей и над ней вьется вертолет. там на носу сидела женщина средних лет похожая на еврейку а на руках у нее мальчик лет трех.'

In [18]:
text

'4 апреля 1984 года. Вчера в кино. Сплошь военные фильмы. Один очень хороший где-то в Средиземном море бомбят судно с беженцами. Публику забавляют кадры где пробует уплыть громадный толстенный мужчина а его преследует вертолет. сперва мы видим как он по-дельфиньи бултыхается в воде потом видим его с вертолета через прицел потом он весь продырявлен и море вокруг него розовое и сразу тонет словно через дыры набрал воды, когда он пошел на дно зрители загоготали. Потом шлюпка полная детей и над ней вьется вертолет. там на носу сидела женщина средних лет похожая на еврейку а на руках у нее мальчик лет трех.'

Передадим текст нашей библиотеке. Назовем этот объект переменной doc.

In [4]:
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

Проведем сегментацию, разбивая текст на токены. Распечатаем первые три.

In [5]:
doc.segment(segmenter)
display(doc.tokens[:3])

[DocToken(stop=1, text='4'),
 DocToken(start=2, stop=8, text='апреля'),
 DocToken(start=9, stop=13, text='1984')]

Разобьем текст на предложения. Распечатаем первые три.

In [6]:
display(doc.sents[:3])

[DocSent(stop=19, text='4 апреля 1984 года.', tokens=[...]),
 DocSent(start=20, stop=33, text='Вчера в кино.', tokens=[...]),
 DocSent(start=34, stop=56, text='Сплошь военные фильмы.', tokens=[...])]

Библиотека **natasha** позволяет делать морфологически разбор слов. Посмотрим на разбор первых пяти слов.
У каждого слова определяется часть речи.

In [7]:
doc.tag_morph(morph_tagger)
display(doc.tokens[:5])

[DocToken(stop=1, text='4', pos='ADJ'),
 DocToken(start=2, stop=8, text='апреля', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=9, stop=13, text='1984', pos='ADJ'),
 DocToken(start=14, stop=18, text='года', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=18, stop=19, text='.', pos='PUNCT')]

Библиотека **natasha** умеет лемматизировать слова. Библиотека решает задачу лемматизации, использует Pymorphy2 и результаты морфологического разбора.
Выведем те, которые отличаются от начальной формы.

In [8]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)
{_.text: _.lemma for _ in doc.tokens if _.text.lower() != _.lemma}

{'апреля': 'апрель',
 'года': 'год',
 'военные': 'военный',
 'фильмы': 'фильм',
 'Средиземном': 'средиземный',
 'бомбят': 'бомбить',
 'беженцами': 'беженец',
 'Публику': 'публика',
 'забавляют': 'забавлять',
 'кадры': 'кадр',
 'пробует': 'пробовать',
 'его': 'он',
 'преследует': 'преследовать',
 'видим': 'видеть',
 'бултыхается': 'бултыхаться',
 'воде': 'вода',
 'вертолета': 'вертолет',
 'продырявлен': 'продырявить',
 'него': 'он',
 'розовое': 'розовый',
 'тонет': 'тонуть',
 'дыры': 'дыра',
 'набрал': 'набрать',
 'воды': 'вода',
 'пошел': 'пойти',
 'зрители': 'зритель',
 'загоготали': 'загоготать',
 'полная': 'полный',
 'детей': 'ребенок',
 'ней': 'она',
 'вьется': 'виться',
 'носу': 'нос',
 'сидела': 'сидеть',
 'средних': 'средний',
 'лет': 'год',
 'похожая': 'похожий',
 'еврейку': 'еврейка',
 'руках': 'рука',
 'нее': 'она',
 'трех': 'три'}

Библиотека умеет работать с именами собственными, приводя слова и целые словосочетания к нормальной форме.
Для этого **natasha** использует результаты синтаксического разбора, учитывает связи между словами, нормализует именованные сущности.


In [9]:
for span in doc.spans:
    span.normalize(morph_vocab)
    
{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{'Средиземном море': 'Средиземное море'}

Библиотека умеет извлекать даты из текста.
А так же имена с помощью NamesExtractor, валюты с помощью MoneyExtractor и адреса с помощью AddrExtractor.

In [10]:
matches = dates_extractor(text)
facts = [i.fact.as_json for i in matches]
facts

[OrderedDict([('year', 1984), ('month', 4), ('day', 4)])]